In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import torch

## ViT

In [2]:
from src.models.vit import vit_base_patch16

In [6]:
wts = "/data/pulkitag/models/ankile/furniture-diffusion/mae/mae_pretrain_vit_base.pth"

state_dict = torch.load(wts, map_location="cpu")

In [8]:
state_dict.keys()

dict_keys(['model'])

In [4]:
vit = vit_base_patch16()

In [10]:
vit.load_state_dict(state_dict["model"], strict=False)

_IncompatibleKeys(missing_keys=['head.weight', 'head.bias'], unexpected_keys=[])

In [11]:
vit.forward_features(torch.randn(1, 3, 224, 224)).shape

torch.Size([1, 768])

## Spatial SoftMax Resnet

In [21]:
from src.models.vision import SpatialSoftmaxEncoder

/data/scratch/ankile/miniconda3/envs/rlgpu/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
enc = SpatialSoftmaxEncoder(
    device="cpu",
    freeze=False,
    use_groupnorm=True,
)

/data/scratch/ankile/miniconda3/envs/rlgpu/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/data/scratch/ankile/miniconda3/envs/rlgpu/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [27]:
img = torch.randint(0, 255, (1, 224, 224, 3))

In [28]:
enc(img).mean()

tensor(-0.0005, grad_fn=<MeanBackward0>)